In [ ]:
import json
import csv
import os

# Define paths to the JSON input and the output CSV
json_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\pacz.json"
csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
csv_file_name = "paczkomaty.csv"
csv_file_path = os.path.join(csv_folder_path, csv_file_name)

# Load the JSON file
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

parcel_lockers = []      # List to store parsed parcel locker data
all_tag_keys = set()     # Set to collect all unique tag keys across lockers

# Iterate through each element in the JSON
for element in data.get('elements', []):
    # Filter only parcel lockers
    if element.get('tags', {}).get('amenity') == 'parcel_locker':
        # Initialize locker info with ID and type
        locker_data = {
            'id': element.get('id'),
            'type': element.get('type')
        }

        # Extract latitude and longitude
        if element.get('type') == 'node':
            # For 'node' types, coordinates are direct
            locker_data['latitude'] = element.get('lat')
            locker_data['longitude'] = element.get('lon')
        else:
            # For other types (e.g., 'way'), use 'center' if available
            locker_data['latitude'] = element.get('lat')
            locker_data['longitude'] = element.get('lon')
            if 'center' in element:
                locker_data['latitude'] = element['center'].get('lat')
                locker_data['longitude'] = element['center'].get('lon')

        # Extract all available tags (metadata) and collect tag keys
        if 'tags' in element:
            for key, value in element['tags'].items():
                locker_data[key] = value
                all_tag_keys.add(key)

        # Add processed locker data to the list
        parcel_lockers.append(locker_data)

# Write results to CSV if any lockers were found
if parcel_lockers:
    base_headers = ['id', 'type', 'latitude', 'longitude']  # Basic columns
    sorted_tag_keys = sorted(list(all_tag_keys))            # Sorted additional metadata columns

    # Combine base headers with unique tag keys
    final_headers = base_headers[:]
    for key in sorted_tag_keys:
        if key not in final_headers:
            final_headers.append(key)

    # Write to CSV
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=final_headers, extrasaction='ignore')
        writer.writeheader()           # Write column headers
        writer.writerows(parcel_lockers)  # Write each row

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\pacz.json
Przetwarzanie elementów...
Znaleziono 19646 paczkomatów.
Zapisywanie danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv


In [ ]:
import csv
import os

# Define input and output file paths
input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

# Define the final columns we want in the output CSV
final_columns = ['id', 'latitude', 'longitude', 'brand']
final_data = []  # List to hold processed rows

# Open and read the input CSV
with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    if reader.fieldnames:  # Check if headers exist
        for row in reader:
            processed_row = {}
            for col in final_columns:
                # Get the value for each column we're interested in, or an empty string if missing
                processed_row[col] = row.get(col, '') 
            final_data.append(processed_row)  # Add to the final list

# Write the filtered data to the output CSV
if final_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=final_columns)
        writer.writeheader()           # Write column headers
        writer.writerows(final_data)   # Write each processed row

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\paczkomaty.csv
Przetworzono 19646 wierszy.
Zapisywanie danych do finalnego pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv


In [ ]:
import csv
import os

# Define input and output file paths
input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final1.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

# Specify which columns to include in the final output
output_columns = ['id', 'latitude', 'longitude', 'brand']
transformed_data = []  # List to store processed rows

# Open and read the input CSV file
with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)

    if reader.fieldnames:  # Check if the file has headers
        for row in reader:
            original_brand = row.get('brand', '').strip()  # Get and clean up the 'brand' field
            modified_brand = original_brand  # Default to original unless we recognize it

            # Normalize the brand names
            if "InPost" in original_brand:
                modified_brand = "Paczkomat InPost"
            elif "DHL" in original_brand:
                modified_brand = "DHL BOX 24/7"
            elif "DPD" in original_brand:
                modified_brand = "DPD Pickup"
            elif original_brand == "Allegro One Box":
                modified_brand = "Allegro One Box"
            elif original_brand == "Orlen Paczka":
                modified_brand = "Orlen Paczka"

            # Build a new dictionary with the desired fields
            transformed_row = {
                'id': row.get('id'),
                'latitude': row.get('latitude'),
                'longitude': row.get('longitude'),
                'brand': modified_brand
            }

            # Add the cleaned and transformed row to our output list
            transformed_data.append(transformed_row)

# Write the cleaned data to the output CSV file
if transformed_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=output_columns)
        writer.writeheader()           # Write the column headers first
        writer.writerows(transformed_data)  # Then write all the cleaned rows

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\final.csv
Przetworzono 19646 wierszy.
Zapisywanie zmodyfikowanych danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv


In [ ]:
import csv
import os

# Define the input and output file paths
input_csv_file_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv"
output_csv_folder_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
output_csv_file_name = "final2.csv"
output_csv_file_path = os.path.join(output_csv_folder_path, output_csv_file_name)

# Define the desired columns in the output
output_columns = ['id', 'latitude', 'longitude', 'brand']

# Define brand keywords that should be excluded
brands_to_remove = ["Paczkomat", "SwipBox"]

# List to store rows that pass the filter
filtered_data = []

# Open and read the input CSV file
with open(input_csv_file_path, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)

    if reader.fieldnames:  # Ensure file has headers
        for row in reader:
            current_brand = row.get('brand', '').strip()

            # Filter out rows whose brand matches any in brands_to_remove
            if current_brand not in brands_to_remove:
                filtered_data.append(row)

# If any rows passed the filter, write them to the new CSV
if filtered_data:
    with open(output_csv_file_path, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=output_columns)
        writer.writeheader()           # Write column headers
        writer.writerows(filtered_data)  # Write filtered rows

Wczytywanie danych z pliku: C:\Users\anton\OneDrive\Pulpit\Mastercard\final1.csv
Usunięto 4 wierszy.
Pozostało 19642 wierszy do zapisania.
Zapisywanie przefiltrowanych danych do pliku CSV: C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv
Dane zostały pomyślnie zapisane do C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv


In [ ]:
import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon
import os
import time
import math

# Ścieżki do plików wejściowych i wyjściowych
base_path = r"C:\Users\anton\OneDrive\Pulpit\Mastercard"
grid_file_path = os.path.join(base_path, "better_coords.csv")  # Plik z punktami siatki
lockers_file_path = os.path.join(base_path, "final2.csv")      # Plik z paczkomatami
output_file_path = os.path.join(base_path, "grid_2km_with_locker_counts.csv")  # Wynik

# Stałe do przeliczania km na stopnie geograficzne
HALF_SIDE_KM = 1.0  # Połowa boku kwadratu (pełny bok = 2 km)
KM_PER_DEGREE_LAT = 111.132  # Stała odległość km/° szerokości
KM_PER_DEGREE_LON_BASE = 111.320  # Bazowa wartość dla długości geograficznej

# Funkcja do przeliczenia km na stopnie szerokości
def calculate_delta_lat_degrees(km):
    return km / KM_PER_DEGREE_LAT

# Funkcja do przeliczenia km na stopnie długości (z uwzględnieniem szerokości geograficznej)
def calculate_delta_lon_degrees(km, center_lat_degrees):
    center_lat_radians = math.radians(center_lat_degrees)
    cos_lat = math.cos(center_lat_radians)
    if abs(cos_lat) < 1e-9:
        return float('inf')
    km_per_degree_lon_at_lat = KM_PER_DEGREE_LON_BASE * cos_lat
    if km_per_degree_lon_at_lat == 0:
        return float('inf')
    return km / km_per_degree_lon_at_lat

# Krok 1: Wczytanie danych o paczkomatach
print("Krok 1: Wczytywanie paczkomatów...")
lockers_df = pd.read_csv(lockers_file_path, usecols=['id', 'latitude', 'longitude', 'brand'], dtype={'id': str})
lockers_df.dropna(subset=['latitude', 'longitude', 'brand'], inplace=True)
lockers_df['brand'] = lockers_df['brand'].astype(str).str.strip()
lockers_df = lockers_df[lockers_df['brand'] != '']
lockers_df['latitude'] = pd.to_numeric(lockers_df['latitude'], errors='coerce')
lockers_df['longitude'] = pd.to_numeric(lockers_df['longitude'], errors='coerce')
lockers_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Konwersja paczkomatów na obiekty geometryczne typu Point
geometry_lockers = [Point(xy) for xy in zip(lockers_df['longitude'], lockers_df['latitude'])]
lockers_gdf = geopandas.GeoDataFrame(lockers_df, geometry=geometry_lockers, crs="EPSG:4326")
print(f"Wczytano {len(lockers_gdf)} paczkomatów.")

# Jeśli brak paczkomatów – zakończ program
if lockers_gdf.empty:
    print("Brak danych o paczkomatach. Przerywam.")
    exit()

# Krok 2: Wczytanie punktów siatki
print("Krok 2: Wczytywanie siatki i tworzenie poligonów...")
grid_df_cols_to_use = ['longitude', 'latitude', 'geometry']
grid_df = pd.read_csv(grid_file_path, usecols=grid_df_cols_to_use)
grid_df.dropna(subset=['latitude', 'longitude'], inplace=True)
grid_df['latitude'] = pd.to_numeric(grid_df['latitude'], errors='coerce')
grid_df['longitude'] = pd.to_numeric(grid_df['longitude'], errors='coerce')
grid_df.dropna(subset=['latitude', 'longitude'], inplace=True)
grid_df.reset_index(drop=True, inplace=True)
grid_df['grid_id'] = grid_df.index  # Nadanie unikalnego ID każdemu punktowi siatki

# Jeśli brak punktów siatki – zakończ program
if grid_df.empty:
    print("Brak danych siatki. Przerywam.")
    exit()

# Tworzenie kwadratowych poligonów wokół każdego punktu siatki
print(f"Wczytano {len(grid_df)} środków kwadratów. Tworzenie poligonów...")
grid_polygons = []
total_grids = len(grid_df)
for index, row in grid_df.iterrows():
    center_lon = row['longitude']
    center_lat = row['latitude']
    delta_lat_deg = calculate_delta_lat_degrees(HALF_SIDE_KM)
    delta_lon_deg = calculate_delta_lon_degrees(HALF_SIDE_KM, center_lat)

    # Oblicz rogi kwadratu 2x2km
    min_lon, max_lon = center_lon - delta_lon_deg, center_lon + delta_lon_deg
    min_lat, max_lat = center_lat - delta_lat_deg, center_lat + delta_lat_deg

    # Utwórz poligon kwadratu
    polygon = Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat), (min_lon, min_lat)])
    grid_polygons.append(polygon)

    # Wydruk postępu
    if (index + 1) % 10000 == 0 or (index + 1) == total_grids:
        print(f"Utworzono geometrię dla {index + 1}/{total_grids} kwadratów...")

# Utworzenie GeoDataFrame z siatki
grid_gdf = geopandas.GeoDataFrame(grid_df, geometry=grid_polygons, crs="EPSG:4326")
print(f"Utworzono {len(grid_gdf)} poligonów.")

# Krok 3: Łączenie przestrzenne (spatial join) – sprawdzenie, które paczkomaty mieszczą się w którym kwadracie
print("Krok 3: Łączenie przestrzenne...")
start_sjoin_time = time.time()
joined_gdf = geopandas.sjoin(lockers_gdf, grid_gdf, how="inner", predicate="within", lsuffix="locker", rsuffix="grid")
print(f"Łączenie przestrzenne zakończone w {time.time() - start_sjoin_time:.2f}s. Znaleziono {len(joined_gdf)} paczkomatów w kwadratach.")

# Krok 4: Agregacja wyników – zliczanie paczkomatów w każdym kwadracie wg marki
if joined_gdf.empty:
    # Brak trafień – utwórz pusty DataFrame z zerami
    result_df = grid_df.copy()
    unique_brands_from_lockers = [str(b).strip() for b in lockers_gdf['brand'].unique() if str(b).strip()]
    for brand_name in unique_brands_from_lockers:
        result_df[brand_name] = 0
else:
    print("Krok 4: Agregacja...")
    group_column = 'grid_id'
    count_column = 'id'
    
    # Grupowanie po grid_id i brand, liczenie liczby paczkomatów
    counts_df = joined_gdf.groupby([group_column, 'brand'])[count_column].count().unstack(fill_value=0)
    
    # Połączenie z oryginalną siatką
    result_df = grid_df.merge(counts_df, on='grid_id', how='left')
    
    # Uzupełnianie brakujących kolumn o zera
    brand_columns_to_fill = [str(b).strip() for b in lockers_gdf['brand'].unique() if str(b).strip()]
    for col in brand_columns_to_fill:
        if col not in result_df.columns:
            result_df[col] = 0
        else:
            result_df[col] = result_df[col].fillna(0).astype(int)
    print("Agregacja zakończona.")

# Krok 5: Zapis wyników do pliku CSV
print("Krok 5: Zapisywanie wyników...")
cols_to_save = list(result_df.columns)

# Jeśli kolumna 'geometry' zawiera obiekty zamiast tekstu – nie zapisujemy jej
if 'geometry' in cols_to_save and not all(isinstance(g, str) for g in result_df['geometry'].dropna()):
    cols_to_save.remove('geometry')

result_df[cols_to_save].to_csv(output_file_path, index=False, encoding='utf-8')
print(f"Wyniki zapisano do {output_file_path}")


Krok 1: Wczytywanie danych o paczkomatach z C:\Users\anton\OneDrive\Pulpit\Mastercard\final2.csv...
Wczytano 19446 paczkomatów jako GeoDataFrame.
Czas wczytywania paczkomatów: 3.98s.

Krok 2: Wczytywanie siatki z C:\Users\anton\OneDrive\Pulpit\Mastercard\better_coords.csv i tworzenie poligonów (2km x 2km)...
Wczytano 77273 środków kwadratów z pliku siatki.
Tworzenie geometrii poligonów dla kwadratów (to może chwilę potrwać)...
Utworzono geometrię dla 5000/77273 kwadratów...
Utworzono geometrię dla 10000/77273 kwadratów...
Utworzono geometrię dla 15000/77273 kwadratów...
Utworzono geometrię dla 20000/77273 kwadratów...
Utworzono geometrię dla 25000/77273 kwadratów...
Utworzono geometrię dla 30000/77273 kwadratów...
Utworzono geometrię dla 35000/77273 kwadratów...
Utworzono geometrię dla 40000/77273 kwadratów...
Utworzono geometrię dla 45000/77273 kwadratów...
Utworzono geometrię dla 50000/77273 kwadratów...
Utworzono geometrię dla 55000/77273 kwadratów...
Utworzono geometrię dla 60000/7